## データ探索概要
- 純粋なカテゴリカル変数でかつ、カテゴリ数の少ない`item_condition_id`, `shipping`はダミー変数化
- textで記載されている特徴量である`name`, `item_description`、 `category_name`は基本的には下記の中から選択
    - CountVectorizer or TfidfVectorizer
    - Cleaningの有無
    - ngramの範囲
    - binary表現有無


|特徴量 |EDA概要| 方針 |  
|------|---------------|----|
| item_condition_id | 5値カテゴリカル変数。 | ダミー変数化 | 
| shipping | 2値のカテゴリカル変数。 | ダミー変数化 |
| name |1,225,273カテゴリあり、最大17語、7語とかが多そう| tf-idf| 
| category_name| 1,287カテゴリあるのでone-hotはつらそう。<br> general_cat, subcat1, subcat2へ分解。それぞれがせいぜい3wordくらい　| それぞれをcountvetor化| 
| brand_name | 4809カテゴリあり、最大7語、だいたいが1、2語　| Label binarize | 
| item_description| 最大245語の文章。 <br> Not described yetなどの定型文もある。 |  tf-idf | 
| name2 | `name`+`brand_name` <br> 1stで使われていた特徴量。 | Tfidf(max_features=100000, token_pattern='\w+') | 
| text | `item_description`+`name+category_name` <br> 1stで使われていた特徴量。 | Tfidf(max_features=100000, token_pattern='\w+', ngram_range=(1, 2)) | 

In [1]:
from contextlib import contextmanager
from functools import partial
from operator import itemgetter
from pathlib import Path
from multiprocessing.pool import ThreadPool
import time
from typing import List, Dict

import keras as ks
import pandas as pd
import numpy as np
import tensorflow as tf
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer as Tfidf
from sklearn.pipeline import make_pipeline, make_union, Pipeline
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.metrics import mean_squared_log_error
from sklearn.model_selection import KFold

DATADIR = Path('./input')

tr_path = DATADIR / 'train.tsv'
test_path = DATADIR / 'test.tsv'

train_cols = ['name', 'item_condition_id', 'category_name', 'brand_name','shipping', 'item_description', 'price']
test_cols =  ['name', 'item_condition_id', 'category_name', 'brand_name','shipping', 'item_description']

train = pd.read_csv(tr_path, sep='\t', usecols=train_cols, parse_dates=True, keep_date_col=True)
test = pd.read_csv(test_path, sep='\t', usecols=test_cols, parse_dates=True, keep_date_col=True)

y_scaler = StandardScaler()

Using TensorFlow backend.


## 処理時間の管理
- contextmanagerを使うことでwith句で囲ってやるだけで、処理時間が表示されるようになって便利。

In [2]:
@contextmanager
def timer(name):
    t0 = time.time()
    yield
    print("{} done in {:.0f} s".format(name, time.time() - t0))

## 特徴量作成
### preprocessでの特徴量作成 
- テキストに関連する特徴量を工夫している、
- `name`と`brand_name`, `item_description`と`name`と`cateogry_name`を混ぜてtextを作っている。
- `*アスタリスク`はapply関数を呼び出しているときにunpackingの機能として適用されいる。`_split_cat()`で各サンプルごとにリストが返され、これをアンパックして各要素を取り出した上で、zipすることでサンプル横断で各要素(general_cat, subcat_1, subcat_2)を得ている。

In [3]:
def preprocess(df: pd.DataFrame) -> pd.DataFrame:
    """preprocessing for whole dataset
    Args:
        df (pd.DataFrame): original whole data from .tsv file.
    Returns:
        pd.DataFrame: data that text concatinated features added.
    """
    df['name2'] = df['name'].fillna('') + ' ' + df['brand_name'].fillna('')
    df['text'] = (df['item_description'].fillna('') + ' ' + df['name'] + ' ' + df['category_name'].fillna(''))
    df['general_cat'], df['subcat_1'], df['subcat_2'] = zip(*df['category_name'].apply(lambda x: _split_cat(x)))

    return df[['name', 'text', 'shipping', 'item_condition_id', 'name2']]

def _split_cat(text):
    try: return text.split("/")
    except: return ("No Label", "No Label", "No Label")

### on_fieldによるpipeline管理
make_pipeline, FunctionTransformer, itemgetterを使って特徴量作成をコンパクトにまとめている。
```python
def on_field(feature: str) -> Pipeline:
    return make_pipeline(FunctionTransformer(itemgetter(feature), validate=False))
```
- make_pipeline(): 処理のパイプラインを作成する
- FunctionTransformer(): すべての要素に関数を適用する
- itemgetter(f): 今回の使われ方だと指定されたfieldのpd.Seriesを返す。e.g. itemgetter('name') -> df['name']。下記と等価。

```python
def itemgetter(item):
    def g(obj):
        return obj[item]
    return g
```

In [4]:
def on_field(feature: str, *vectorizer) -> Pipeline:
    """make a pipeline for vectorization with the specified feature.
    Args:
        feature (str): column name for the preprocessing target feature.
        vectorizer : functions for vectorization.
    Return: A pipeline for preprocessing.
    """
    return make_pipeline(FunctionTransformer(itemgetter(feature), validate=False), *vectorizer)

def to_records(df: pd.DataFrame) -> List[Dict]:
    """convert the DataFrame to a dictionary.
    Args:
        df (pd.DataFrame): data.
    Returns:
        List of dictionaries. Keys are column names, values are values of cell.
    """
    return df.to_dict(orient='records')

### make_unionによる特徴量ベクトル化の効率化
make_unionで[FeatureUnion](https://scikit-learn.org/stable/modules/generated/sklearn.pipeline.FeatureUnion.html#sklearn.pipeline.FeatureUnion)クラスのvectorizerインスタンスを作っている。このインスタンスは後に`fit_transform`メソッドが呼ばれてtransformersによって前処理された結果がhstackされたnp.ndarray(n_samples, sum_n_components)を返している。特徴量作成に便利そう。さらっとあるn_jobsがnotebook上でもおしゃれに並列化をかませてくれる。

In [5]:
vectorizer = make_union(
    on_field('name', Tfidf(max_features=100000, token_pattern='\w+')),
    on_field('text', Tfidf(max_features=100000, token_pattern='\w+', ngram_range=(1, 2))),
    on_field(['shipping', 'item_condition_id'], FunctionTransformer(to_records, validate=False), DictVectorizer()),n_jobs=4)

### Train/Valid分割 
- KFoldでn_split=20としているが、next()でiteretorを1回しか呼び出していないので実質20分の1だけvalidにsubsamplingしてきただけ？

### 説明変数`price`の前処理
- log(y+1)変換:y_trainをポアソン回帰的にじゃなくてlog(y+1)変換したものに対してfitするようにしている。後の学習時に`mean squared error`をfitしに行っているので評価関数的に整合性取れている。
- スケールを平均0、分散1に変更している。2stage制なので2ndのテストデータがどのようなものが来るかわからないから？あとで読む。

In [6]:
with timer('train valid data split'):
    train = train[train['price'] > 0].reset_index(drop=True) # reset_indexないとilocで変な値取ってくる
    cv = KFold(n_splits=20, shuffle=True, random_state=42)
    train_ids, valid_ids = next(cv.split(train))
    train, valid = train.iloc[train_ids], train.iloc[valid_ids]

with timer('process train'):  
    y_train = y_scaler.fit_transform(np.log1p(train['price'].values.reshape(-1, 1)))
    X_train = vectorizer.fit_transform(preprocess(train)).astype(np.float32)
    print('X_train: {} of {}'.format(X_train.shape, X_train.dtype))
    del train
    
with timer('process valid'):
    y_valid = valid['price']
    X_valid = vectorizer.transform(preprocess(valid)).astype(np.float32)
    print('X_valid: {} of {}'.format(X_valid.shape, X_valid.dtype))
    del valid

train valid data split done in 1 s
X_train: (1407577, 200002) of float32
process train done in 214 s
X_valid: (74084, 200002) of float32
process valid done in 46 s


## 学習
### モデル
- モデル自体は`fit_predict`でtensorflowでの4層パーセプトロン。
- vectorizerがsparse matrixを返してくるので`sparse=True`とされている。
- batch_sizeが学習が進むと徐々に大きくなる。最初小さいバッチサイズで進めて、さくさく勾配を下させて、除々に速度を犠牲にしてデータ全体に最適化するようにしている。学習率大→小へと進めるのと似た意味合い。

In [7]:
def fit_predict(xs, y_train) -> np.ndarray:
    X_train, X_test = xs
    config = tf.ConfigProto(
        intra_op_parallelism_threads=1, use_per_session_threads=1, inter_op_parallelism_threads=1)
    with tf.Session(graph=tf.Graph(), config=config) as sess, timer('fit_predict'):
        ks.backend.set_session(sess)
        model_in = ks.Input(shape=(X_train.shape[1],), dtype='float32', sparse=True)
        out = ks.layers.Dense(192, activation='relu')(model_in)
        out = ks.layers.Dense(64, activation='relu')(out)
        out = ks.layers.Dense(64, activation='relu')(out)
        out = ks.layers.Dense(1)(out)
        model = ks.Model(model_in, out)
        model.compile(loss='mean_squared_error', optimizer=ks.optimizers.Adam(lr=3e-3))
        for i in range(3):
            with timer('epoch {}'.format(i+1)):
                model.fit(x=X_train, y=y_train, batch_size=2**(11 + i), epochs=1, verbose=0)
        return model.predict(X_test)[:, 0]

## 4つのNNモデルのアンサンブル＆並列学習
- NNを活用する際は確率的勾配法を活用していることから同じモデルでもアンサンブルした方が良い？
- np.boolとすることでzeroとnon-zeroで2値化されたデータを作った。意味合い的にはtf-idfからbinary CountVectorizeしていることになる。

In [8]:
with ThreadPool(processes=4) as pool:
    Xb_train, Xb_valid = [x.astype(np.bool).astype(np.float32) for x in [X_train, X_valid]]
    xs = [[Xb_train, Xb_valid], [X_train, X_valid]] * 2
    y_pred = np.mean(pool.map(partial(fit_predict, y_train=y_train), xs), axis=0)
    y_pred = np.expm1(y_scaler.inverse_transform(y_pred.reshape(-1, 1))[:, 0])
    print('Valid RMSLE: {:.4f}'.format(np.sqrt(mean_squared_log_error(y_valid, y_pred))))

Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use tf.cast instead.
epoch 1 done in 511 s
epoch 1 done in 556 s
epoch 1 done in 556 s
epoch 1 done in 557 s
epoch 2 done in 297 s
epoch 2 done in 273 s
epoch 2 done in 295 s
epoch 2 done in 295 s
epoch 3 done in 171 s
fit_predict done in 985 s
epoch 3 done in 160 s
fit_predict done in 996 s
epoch 3 done in 165 s
epoch 3 done in 166 s
fit_predict done in 1024 s
fit_predict done in 1025 s
Valid RMSLE: 0.3936
